In [18]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html
import pandas as pd


# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret='C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624',
)



In [19]:
print(reddit.read_only)

True


In [20]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

Some test commands
Proof
[Recruiting] Arcane Angels | #LYUQG8CL | TH16 | Lvl 27 | Friendly & Engaged | Arcane Alliance
Test Speciosa-2
[Recruiting] Arcane Angels | #LYUQG8CL | TH16 | Lvl 27 | Friendly & Engaged | Arcane Alliance
[Recruiting] Arcane Angels | #LYUQG8CL | TH16 | Lvl 27 | Friendly & Engaged | Arcane Alliance
Tester 2
Hello, I'm testing
test
[Recruiting] Arcane Angels | #LYUQG8CL | TH16 | Lvl 27 | Friendly & Engaged | Arcane Alliance


In [21]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)

wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [29]:

stockCSV = pd.read_csv('stock_info.csv')
stocksAbbrev = stockCSV['Ticker'].tolist() 
stocksAbbrev #ABBREVIATIONS
stockFull = stockCSV['Name'].tolist() 
stockFull #FUll NAMES


['Agilent Technologies',
 'Alcoa Inc.',
 "Aaron's Inc",
 'American Assets Trust',
 'Advantage Oil & Gas Ltd',
 'Alliance Capital Management L.P.',
 'Abb Ltd',
 'Abbvie Inc. Common Stock',
 'Amerisourcebergen Corporation',
 'Abm Industries Incorporated',
 'Arbor Realty Trust',
 'Arbor Realty Trust Preferred Se',
 'Arbor Realty Trust Cumulative R',
 'Abbot Laboratories',
 'Companhia De Bebidas - Ambev',
 'Companhia De Bebidas - Ambev',
 'Associated Banc-Corp Depositary',
 'Barrick Gold Corp.',
 'Ace Limited',
 'Alliancebernstein Income Fund Inc',
 'Aluminum Corporation of China Ltd.',
 'Arch Coal',
 'Aecom Technology Corp.',
 'Access Midstream Partners L.P.',
 'Accenture Plc.',
 'Amcol International Corp.',
 'Avenue Income Credit Strategies',
 'Ares Commercial Real Estate Cor',
 'Actavis Inc.',
 'Active Network Inc',
 'Accuride Corporation New',
 'Agree Realty Corp.',
 'Archer-Daniels-Midland Company',
 'Alliance Data Systems Corp.',
 'Adt Corporation Common Stock W',
 'Adams Express Com

In [35]:

postData = []
# Fetch the top 1000hot posts in the 'wallstreetbets' subreddit
for submission in subreddit.hot(limit=1000):
        if(submission.selftext == ''): #skip any posts that contain no text
            continue
        elif(submission.selftext == stocksAbbrev or submission.selftext == stockFull):
            postData.append({
                'title': submission.title,
                'date': submission.created_utc,
                'score': submission.score,  # upvotes
                'id': submission.id,
                'url': submission.url,
                'comms_num': submission.num_comments,
                'body': submission.selftext
        })
        else:
            continue
        

In [23]:
import pandas as pd

df = pd.DataFrame(postData)
df



,title,date,score,id,url,comms_num,body
0,"What Are Your Moves Tomorrow, October 31, 2024",1.730318e+09,144,1gfvose,https://www.reddit.com/r/wallstreetbets/commen...,4901,This post contains content not supported on ol...
1,+646K on $SMCI Lotto,1.730298e+09,4990,1gfntw1,https://i.redd.it/zytbpjtplwxd1.jpeg,605,I can post the video if me logging in for proo...
2,THANKS REDDIT 15k to 400k,1.730302e+09,2498,1gfp2mn,https://www.reddit.com/gallery/1gfp2mn,184,got rddt 90c 11/1\nand 85c 1/15 earlier in the...
3,PLTR CEO sold $254 Million worth of shares. Ch...,1.730307e+09,585,1gfr5v7,https://www.reddit.com/r/wallstreetbets/commen...,154,Puts?\n\nhttps://preview.redd.it/p474ecisfxxd1...
4,$94k to $1.2M $RDDT Earnings Play,1.730320e+09,245,1gfwdcg,https://www.reddit.com/r/wallstreetbets/commen...,69,I'm getting tired of this fucking FOMO man. I'...
...,...,...,...,...,...,...,...
375,Robinhood performance - Help understand its mo...,1.729768e+09,12,1gazuqq,https://www.reddit.com/r/wallstreetbets/commen...,10,"Hi all, you might have noticed the strong perf..."
376,It is very clear to me that WSB goes full rega...,1.729725e+09,88,1gao0i1,https://www.reddit.com/r/wallstreetbets/commen...,260,Never go full regard. Stop letting your hate f...
377,L3Harris earnings,1.729784e+09,4,1gb5e4x,https://www.reddit.com/r/wallstreetbets/commen...,14,Anyone playing this? Implied move not hugely e...
378,UBER is 100% going to crush earnings,1.729102e+09,406,1g55vs8,https://www.reddit.com/r/wallstreetbets/commen...,205,The crux of this thesis is well explained in t...


In [34]:
df.to_csv('redditData.csv', index=False) #save to csv file